# Generator based Coroutine

## basic behavior

In [ ]:
def simple_coroutine(): 
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [ ]:
my_coro = simple_coroutine()

In [ ]:
my_coro

In [ ]:
type(my_coro)

In [ ]:
next(my_coro)

In [ ]:
my_coro.send(30)

## Coroutine (actually generator) Status (Life cycle)

In [ ]:
def simple_coro2(a): 
    print('-> Started: a =', a)
    b= yield a
    print('-> Received: b =', b)
    c = yield a+b
    print('-> Received: c =', c)

In [ ]:
my_coro2 = simple_coro2(14)

In [ ]:
from inspect import getgeneratorstate

In [ ]:
getgeneratorstate(my_coro2)

In [ ]:
my_coro2.send(12)

In [ ]:
next(my_coro2)

In [ ]:
getgeneratorstate(my_coro2)

In [ ]:
my_coro2.send(50)

In [ ]:
getgeneratorstate(my_coro2)

In [ ]:
my_coro2.send(1050)

In [ ]:
getgeneratorstate(my_coro2)

* 코루틴 동작 방식 (그림 출처 : fluent python(2015))

![gen_coro_flow](./images/gen_coro_flow.png)

## Sample : 이동평균을 계산하는 코루틴

In [ ]:
def averager():
    total = 0.0
    count = 0
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count


In [ ]:
coro_avg = averager()
next(coro_avg)

In [ ]:
getgeneratorstate(coro_avg)

In [ ]:
coro_avg.send(None)

In [ ]:
getgeneratorstate(coro_avg)

## 코루틴의 종료와 예외처리

In [ ]:
coro_avg = averager()
next(coro_avg)
getgeneratorstate(coro_avg)

In [ ]:
coro_avg.send(50)

In [ ]:
coro_avg.send(120)

In [ ]:
coro_avg.close()

In [ ]:
getgeneratorstate(coro_avg)

### 코루틴 예외처리를 방법을 설명하기 위한 제너레이터

In [ ]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling(): 
    print('-> coroutine started') 
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)
getgeneratorstate(exc_coro)

In [ ]:
exc_coro.send(11)

In [ ]:
exc_coro.send(20)

In [ ]:
exc_coro.close()

In [ ]:
getgeneratorstate(exc_coro)

#### 예외 발생시키기

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)
getgeneratorstate(exc_coro)

In [ ]:
exc_coro.throw(DemoException)

In [ ]:
getgeneratorstate(exc_coro)

In [ ]:
exc_coro.throw(ZeroDivisionError)

In [ ]:
getgeneratorstate(exc_coro)

#### 코루틴 종료시 어떤작업을 실행하려면 try-finally 구문 사용

In [ ]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_finally():
    print('-> coroutine started') 
    try:
        while True: 
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...') 
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [ ]:
exc_coro = demo_finally()
next(exc_coro)
getgeneratorstate(exc_coro)

In [ ]:
exc_coro.send(11)

In [ ]:
exc_coro.throw(DemoException)

In [ ]:
exc_coro.throw(ZeroDivisionError)

## 코루틴에서 값을 반환하기

In [ ]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None 
    while True:
        term = yield average 
        if term is None:
            break
        
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [ ]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

In [ ]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
print(result)

In [ ]:
coro_avg = averager()
next(coro_avg)

In [ ]:
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

In [ ]:
r = coro_avg.close()

In [ ]:
print(r)

## yield from

In [ ]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i


In [ ]:
gen()

In [ ]:
list(gen())

In [ ]:
def gen():
    yield from 'AB'
    yield from range(1,3)


In [ ]:
gen()

In [ ]:
list(gen())

In [ ]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
    def __repr__(self):
        return f"Senetence ({reprlib.repr(self.text)})"

In [ ]:
def sentence_gen():
    yield from iter(Sentence("The old man and sea"))
    yield from Sentence("Crime and Punishment")

In [ ]:
sentence_gen()

In [ ]:
list(sentence_gen())

### generator와 함께 사용

In [ ]:
def main_gen():
    while True:
        yield from averager()

In [ ]:
main = main_gen()

In [ ]:
main

In [ ]:
next(main)

In [ ]:
main.send(30)

In [ ]:
main.send(4)

In [ ]:
main.send(60)

In [ ]:
r = main.send(None)

In [ ]:
def main_gen2():
    while True:
        yield from averager()
        yield from sentence_gen()

In [ ]:
main = main_gen2()
next(main)

In [ ]:
main.send(30)

In [ ]:
next(main)

In [ ]:
next(main)

In [ ]:
main.send(40)

In [ ]:
main.send(40)

In [ ]:
next(main)

In [ ]:
list(main)

### Sample : 가상의 중1 학생 키와 몸무게 평균

In [ ]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


# the delegating generator
def grouper(results, key): 
    while True:
        results[key] = yield from averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key)
        print(f"{key}, {type(group)}, {id(group)}")
        next(group)
        for value in values:
            group.send(value) 
        group.send(None) # important!
        print(results)  # uncomment to debug
    report(results) 
        
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5} averaging {result.average:.2f}{unit}')

data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

![averager](./images/group_averager.png)

In [ ]:
data.items()

In [ ]:
for k, v in data.items():
    print(f"{k = }, { v =}")

# Async & Await 

In [ ]:
def hello():
    print("Hello, Old-Fashioned World!")

hello()

In [ ]:
async def async_hello():
    print("Hello, Asynchronous World!")

async_hello()

In [ ]:
def gen_hello():
    yield 
    print("Hello, Generator World!")

gen_hello()

In [ ]:
coro_hello = async_hello()
#next(coro_hello)
try:
    coro_hello.send(None)
except StopIteration as ex:
    print(ex.value)
    

In [ ]:
import types

@types.coroutine
def lazy_range(n):
    while n >= 0:
        yield n
        n -= 1
    return n
    
async def print_list():
    res = await lazy_range(5)
    print(f"{res = }")

In [ ]:
coro_list = print_list()
try:
    coro_list.send(None)
    while True:
        print("Step", coro_list.send(None))
except StopIteration as ex:
    print(ex.value)

In [ ]:
a = lazy_range(5)
a

### Sentence를 Awaitable로 구현하기

In [ ]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text

    def __await__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
    def __repr__(self):
        return f"Senetence ({reprlib.repr(self.text)})"

In [ ]:
async def get_word():
    await Sentence("the old man and sea")

In [ ]:
word = get_word()
word

In [ ]:
try:
    while True:
        print("Step", word.send(None))
except StopIteration as ex:
    print(ex.value)

In [ ]:
word.send(None)

### Averager를 awitable로 구현하기

In [ ]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

#import types

#@types.coroutine
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [ ]:
await_avg = averager()
await_avg

In [ ]:
await_avg.send(None) # start

In [ ]:
await_avg.send(30)
await_avg.send(40)
await_avg.send(50)

In [ ]:
await_avg.send(None)

In [ ]:
# the delegating generator
async def grouper(results, key): 
    while True:
        #results[key] = yield from averager()
        results[key] = await averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key)
        print(f"{key}, {type(group)}, {id(group)}")
        #next(group)
        group.send(None)
        for value in values:
            group.send(value) 
        group.send(None) # important!
        # print(results)  # uncomment to debug
    report(results) 
        
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5} averaging {result.average:.2f}{unit}')

data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

In [ ]:
group = grouper(results, key)
group

In [ ]:
a = avg([10, 20, 30, 40])
a

In [ ]:
a.send(None)

In [ ]:
a.send(10)
a.send(20)
a.send(30)
a.send(40)

In [ ]:
a.send(None)